In [153]:
import pandas as pd
import os
from blastScripts import make_queries as mq


## Run reciprocal blast

In [154]:

def reciprocal_blastp(database_dir, query_file, out_dirname, proteome_dir, reference_proteome_path):
    databases  = [db for db in os.listdir(database_dir) if db != '.DS_Store']
    proteomes = [p.replace('.faa', '') for p in os.listdir(proteome_dir)]
    for database in databases:
        if database not in proteomes:
            raise Exception(f'{proteome} not present in proteomes')
    os.makedirs(out_dirname)
    forward_results_dir = os.path.join(out_dirname, 'forward_blast_results')
    multi_database_query(database_dir, query_file, forward_results_dir)

    rblast_queries_dir = os.path.join(out_dirname, 'forward_best_hits')
    mq.write_best_hit_queries(proteome_dir, forward_results_dir, rblast_queries_dir)
    
    
    ref_db_name = reference_proteome_path.replace('.faa', '_blast_db')
    #os.makedirs('reference_db')
    reference_db_path = os.path.join(out_dirname, 'reference_db/reference_db')
    print(reference_db_path)
    !makeblastdb -in $reference_proteome_path -dbtype 'prot' -out $reference_db_path
    recip_blast(rblast_queries_dir, reference_db_path, os.path.join(out_dirname,'reciprocal_blast_results'))
    
    
    
query_file = '../Pseudomonas_1_csrA.faa'
database_dir = '../test_blast_db/'
proteome_dir = '../proteomes'
reference_proteome_path = '../P_aeruginosa_PA01.faa'
out_dirname = '../test_rblast_results_Psuedo'
  
reciprocal_blastp(database_dir, query_file, out_dirname, proteome_dir, reference_proteome_path)

NameError: name 'multi_database_query' is not defined

In [155]:
metadata_df =  pd.read_csv('../gut_microbiome/genomes-all_metadata.tsv', sep = '\t')
metadata_df = metadata_df.set_index('Genome')


### get metedata for human gut microbiome \ 
FTP downloads:  
http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v2.0.1/  


In [156]:
### get accessions for species representative
len(set(metadata_df['Species_rep']))
rep_accessions = list(set(metadata_df['Species_rep']))
metadata_df_repr = metadata_df.loc[rep_accessions,:]

In [ ]:
for acc in rep_accessions:
    url = f'https://www.ebi.ac.uk/metagenomics/api/v1/genomes/{acc}/downloads/{acc}.faa'
    out_fp = f'../Proteomes/{acc}.faa'
    !curl $url > $out_fp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  526k  100  526k    0     0   234k      0  0:00:02  0:00:02 --:--:--  234k    0      0      0 --:--:-- --:--:-- --:--:--     0
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  302k  100  302k    0     0   161k      0  0:00:01  0:00:01 --:--:--  161k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1390k  100 1390k    0     0   638k      0  0:00:02  0:00:02 --:--:--  637k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1002k  100 1002k    0     0   442k      0  0:00:02  0:00:02 --:--:--  442k


### Make blast database from each proteome file

In [4]:

def make_blastp_databases(proteome_dir, out_dir):
    """Make a blastp database for each proteome within a given directory"""
    lst_assembly_accessions = [acc.replace('.faa', '') for acc in os.listdir(proteome_dir)]
    for acc in lst_assembly_accessions:
        proteome_path = f'{proteome_dir}/{acc}.faa'
        out = f'{out_dir}/{acc}/{acc}_db'
        !makeblastdb -in $proteome_path -dbtype 'prot' -out $out

In [5]:
#make_blastp_databases('../proteomes', './blast_databases')

### Query each database with the original sequence to which you are finding orthologs  

In [4]:
def multi_database_query(database_dir, query_filepath, out_dir_path):
    """Query each blastp database within the database_dir and make a hits text file in the out_dir"""
        
    if not out_dir_path:
        out_dir_path = './forward_blast_results'
    if not os.path.exists(out_dir_path):
        os.makedirs(out_dir_path)
    for acc in [acc for acc in os.listdir(database_dir) if acc != '.DS_Store']:
        base_filepath = os.path.join(database_dir, f'{acc}/{acc}_db')
        out_filepath = os.path.join(out_dir_path, f'{acc}_blastpOut.txt')
        print(out_filepath)
        !blastp -query $query_file -db $base_filepath -num_descriptions 5 -num_alignments 5 -out $out_filepath
        
# query_file = '../B_subtilis_csra.faa'
# database_dir = '../blast_databases'
# out_dir_path = '../forward_blast_results_1'
# multi_database_query(database_dir, query_file, out_dir_path)    

#### Make individual fasta files from forward blast best hit for rblast queries  

In [17]:
rblast_queries_dir = './forward_best_hits'

from blastScripts import make_queries as mq
mq.write_best_hit_queries('../proteomes/', './forward_blast_results/', rblast_queries_dir)

MGYG000000252 

MGYG000004000 

MGYG000001252 



{'MGYG000003897': {'MGYG000003897_00073': 'MSTLQNILEYLRPIQTHLVDERPIRHILTDSRKLTSPSDSLFFAMRTASGDGHKYIPQLYEHGVRAFFICEPIEPYVERYPDANIVQVQETLRALQQIASHWRMRYDYPVIGITGSNGKTVVKEFLYHLLSGCYRIVRSPKSYNSQLGVPLSILNMEEEHTLAIFEAGISMPMEMLRLQRIIRPTLGILTNIGAAHQENFVSLNQKIEEKLQLFVDADHFVYDADSDEIQRGIDNLGLSDKAIGWSLTPGKAYYHVSYSNSGDGTTTITAQHADESSTVTIPFADQASIQNATHCLCLIGLLPLTTAQRQAVLARFATLEAIEMRLEVKEGQAGNRLINDAYNNDINSLRIALDFQKQRTATSHQQAVIILSDILQSAQLPRDLYKQVGEMIAQYPHTLFIGVGRELCNYQDYFQRSGEGKTAFYETTDQLLADDLLGTISQAEILVKGARQFQFERIVQHLAKQVHETTLEIDLEALVSNLKSYKALLPSETRIIVMAKAQGYGIGAYELAKALEQQDLAALAVALADEGKELRSKGILLPIIVMNPEVEAFEVMTQSRLEPEIYNERILREFARHVERQGLSHYPVHIELDTGMHRTGFTPDRETLEHLAQTLHEVEPLIRVQSIFTHLAAADEPTMSDFTEQQFALYDEGADYLTSQLSYRPLRHVQNTAGIERYNHHHYDMARLGVGLYGISATGSLTLEPVAKLRTTLLQIKTIPDGETIGYGRRGQVAPGGQIGIIPIGYADGYDRRFSCGVGSVMIHDTLCPIVGNVCMDTCMVDLSPLQGKVAEGDEVIIFGVPGLQVSDLAERIGTIPYEVISKLSPRIKRTYYKS'},
 'MGYG000001078': {'MGYG000001078_00311': 'MTTGLYLSAMSQNSGKSLVALGLADSLIKRADRVGFFRPVFDGATIADDPMARLIREHFGLTEEQVGGAVSMTDA

#### Make blast database for B. subtilis proteome

In [18]:
import os
os.listdir(rblast_queries_dir)
rblast_db_path = './GCF_000009045.1/GCF_000009045.1_db'
!makeblastdb -in GCF_000009045.1.faa -dbtype 'prot' -out $rblast_db_path



Building a new DB, current time: 03/16/2023 10:41:53
New DB name:   /Users/jonwinkelman/Trestle_projects/Mukherjee_lab/csrA/Notebook_scripts/GCF_000009045.1/GCF_000009045.1_db
New DB title:  GCF_000009045.1.faa
Sequence type: Protein
Deleted existing Protein BLAST database named /Users/jonwinkelman/Trestle_projects/Mukherjee_lab/csrA/Notebook_scripts/GCF_000009045.1/GCF_000009045.1_db
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 4237 sequences in 0.093555 seconds.




### Reciprocal blast  

In [24]:
def recip_blast(rblast_queries_dir, rblast_db_path, outDir):
    """ """
    
    os.makedirs(outDir)
    query_files = [file for file in os.listdir(rblast_queries_dir) if file.endswith('.faa')]
    for query in query_files:
        outfile_name = query.replace('.faa', '_blastpOut.txt')
        out_path = os.path.join(outDir, f'{outfile_name}')
        query_path  = os.path.join(rblast_queries_dir, query)
        print(query_path,rblast_db_path, out_path, )
        !blastp -query $query_path -db $rblast_db_path -num_descriptions 5 -num_alignments 5 -out $out_path

In [62]:
csrA_prot_ID = 'NP_391417.1'
ortho_dict = {}
for accession, rhit_ID in best_rHits_d.items():
    if rhit_ID == csrA_prot_ID:
        ortho_dict[accession] = rhit_ID


In [150]:

acc_taxid_d = {acc:i for i,acc in enumerate(rep_accessions)}
prot_id_taxid_d = {}
with open('../all_hs_gut_proteomes.faa', 'r') as f:
    for line in f:
        if line.startswith('>'):
            prot_id  = line.split(' ')[0][1:]
            acc = prot_id.split('_')[0]
            prot_id_taxid_d[prot_id] = acc_taxid_d[acc]
            
with open('../TaxID_map.txt', 'w') as f:
    for prot_id, taxid in prot_id_taxid_d.items():
        f.write(f'{prot_id} {taxid}\n')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 2874: invalid start byte

NameError: name 'rblast_queries_dir' is not defined